In [0]:
spark.conf.set(
  "fs.azure.account.key.masterrdp001sta.dfs.core.windows.net",
  "ubrM/TOROC6KRdGhHI04+8tXLRbVdnrsYAIEKoGkiI1LVta7uERw8AgT6YtqbrukLsjX5gJzKcqz+AStS8tZSA=="
)

In [0]:
from pyspark.sql import functions as F

1) Crear tabla dim_weather

In [0]:
#Crear tabla
dim_weather = (
    spark.table("hive_metastore.tfm.datos_brutos_weather")
    .select(
        "STATION", 
        "NAME",
        "LATITUDE",
        "LONGITUDE"
    ).distinct()
)

display(dim_weather.limit(10))

STATION,NAME,LATITUDE,LONGITUDE
USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.68275,-74.16927
USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925
USW00014732,"LAGUARDIA AIRPORT, NY US",40.77945,-73.88027
USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.7639


In [0]:
#Poner todas las columnas en minisculo

for c in dim_weather.columns:
    column = c.lower()
    dim_weather = dim_weather.withColumnRenamed(c, column)

display(dim_weather.limit(100))

station,name,latitude,longitude
USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.68275,-74.16927
USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925
USW00014732,"LAGUARDIA AIRPORT, NY US",40.77945,-73.88027
USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.7639


2) Guardar tabla dim_weather

In [0]:
#Por último, antes de guardar renomear nombres de columnas para que sea más sencillo entender las tablas

dim_weather = (
    dim_weather
    .withColumnRenamed('station','weather_station_id')
)

In [0]:
display(dim_weather.limit(100))

weather_station_id,name,latitude,longitude
USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.68275,-74.16927
USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925
USW00014732,"LAGUARDIA AIRPORT, NY US",40.77945,-73.88027
USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.7639


In [0]:
dim_weather.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tfm.dim_weather")